## 1. 初始化

In [1]:

import pandas as pd
import numpy as np
import xgboost as xgb
from scipy.spatial.distance import cdist
from sklearn.metrics import ndcg_score, average_precision_score
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

\
plt.style.use('ggplot')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 
plt.rcParams['axes.unicode_minus'] = False

## 2. 載入資料

In [2]:
try:
    df_raw = pd.read_csv('../data_after_process/violate_with_type.csv', encoding='utf-8-sig')
    df_coords = pd.read_csv('../data_after_process/unique_locations.csv', encoding='utf-8-sig')
    df_rules = pd.read_csv('../data_after_process/location_rules.csv', encoding='utf-8-sig')
    df_rain = pd.read_csv('../data_after_process/rainfall.csv', encoding='utf-8-sig')
    df_temp = pd.read_csv('../data_after_process/temperature.csv', encoding='utf-8-sig')
except FileNotFoundError as e:
    print(f"找不到檔案: {e}")
    print("請先執行 cluster_locations.py 產生 location_rules.csv")
    exit()
# 建立地點到區域的映射
loc_to_zone = df_rules.set_index('Original_Location')['Zone_ID'].to_dict()
zone_names = df_rules.drop_duplicates('Zone_ID').set_index('Zone_ID')['Zone_Name'].to_dict()

# 處理原始資料
df_raw['Datetime'] = pd.to_datetime(df_raw['舉發日期'], errors='coerce')
df_raw = df_raw.dropna(subset=['Datetime'])
df_raw = df_raw[df_raw['Datetime'].dt.year >= 2023].copy()

# 將地點映射到區域
df_raw['Zone_ID'] = df_raw['違規地點'].map(loc_to_zone)
df_raw = df_raw.dropna(subset=['Zone_ID'])
df_raw['Zone_ID'] = df_raw['Zone_ID'].astype(int)
print(f"篩選後資料: {len(df_raw)} 筆")

篩選後資料: 27139 筆


## 3.建立時間網格 (區域版)

In [3]:
print("\n建立區域時間網格...")

FREQ = '15min'
start_time = df_raw['Datetime'].min().floor('D')
end_time = df_raw['Datetime'].max().ceil('D')
time_index = pd.date_range(start=start_time, end=end_time, freq=FREQ)

# 取締時段
morning_mask = (time_index.hour >= 9) & ((time_index.hour < 11) | ((time_index.hour == 11) & (time_index.minute <= 30)))
afternoon_mask = (time_index.hour >= 13) & ((time_index.hour < 16) | ((time_index.hour == 16) & (time_index.minute <= 30)))
active_time_index = time_index[morning_mask | afternoon_mask]

# 區域列表
zones = sorted(df_raw['Zone_ID'].unique())
print(f"有效時段數: {len(active_time_index)}")
print(f"區域數: {len(zones)}")

# 建立網格
idx = pd.MultiIndex.from_product([active_time_index, zones], names=['Slot_Start', 'Zone_ID'])
df_grid = pd.DataFrame(index=idx).reset_index()

# 計算每個區域每個時段的取締數
df_raw['Slot_Start'] = df_raw['Datetime'].dt.floor(FREQ)
counts = df_raw.groupby(['Slot_Start', 'Zone_ID']).size().reset_index(name='count_in_slot')

df = pd.merge(df_grid, counts, on=['Slot_Start', 'Zone_ID'], how='left')
df['count_in_slot'] = df['count_in_slot'].fillna(0).astype(int)
df = df.sort_values(['Zone_ID', 'Slot_Start']).reset_index(drop=True)


建立區域時間網格...
有效時段數: 27404
區域數: 22


## 4.計算區域中心座標

In [4]:
print("\n計算區域中心座標...")

coords_dict = df_coords.set_index('Location')[['Latitude', 'Longitude']].to_dict('index')

zone_coords = {}
for zone_id in zones:
    zone_locs = df_rules[df_rules['Zone_ID'] == zone_id]['Original_Location'].tolist()
    lats = [coords_dict[loc]['Latitude'] for loc in zone_locs if loc in coords_dict]
    lons = [coords_dict[loc]['Longitude'] for loc in zone_locs if loc in coords_dict]
    if lats and lons:
        zone_coords[zone_id] = {
            'Latitude': np.mean(lats),
            'Longitude': np.mean(lons)
        }

# 計算區域間距離
def haversine(coord1, coord2):
    lat1, lon1 = np.radians(coord1)
    lat2, lon2 = np.radians(coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    return 6371000 * 2 * np.arcsin(np.sqrt(a))

zone_list = [z for z in zones if z in zone_coords]
zone_to_idx = {z: i for i, z in enumerate(zone_list)}
n_zones = len(zone_list)

dist_matrix = np.zeros((n_zones, n_zones))
for i, z1 in enumerate(zone_list):
    for j, z2 in enumerate(zone_list):
        if i != j:
            c1 = (zone_coords[z1]['Latitude'], zone_coords[z1]['Longitude'])
            c2 = (zone_coords[z2]['Latitude'], zone_coords[z2]['Longitude'])
            dist_matrix[i, j] = haversine(c1, c2)

# 建立鄰近區域
neighbor_map = {}
for i, zone in enumerate(zone_list):
    sorted_indices = np.argsort(dist_matrix[i])
    # 取最近的 5 個鄰居 (排除自己)
    neighbor_map[zone] = [zone_list[idx] for idx in sorted_indices[1:6] if dist_matrix[i, idx] > 0]
print("完成")


計算區域中心座標...
完成


## 5. 特徵工程

In [5]:
print("\n進行特徵工程...")

# 處理降水量缺失值
# 取前兩天和後兩天的平均值補全
df_rain['Precipitation'] = df_rain['Precipitation'].replace('T', np.nan).astype(float)
filled = df_rain['Precipitation'].copy()
for i in range(len(df_rain)):
    if pd.isna(df_rain.loc[i, 'Precipitation']):
        # 前兩天 + 後兩天
        neighbors = []
        
        # 前兩天
        for j in range(1, 3):
            if i - j >= 0:
                neighbors.append(df_rain.loc[i-j, 'Precipitation'])
        
        # 後兩天
        for j in range(1, 3):
            if i + j < len(df_rain):
                neighbors.append(df_rain.loc[i+j, 'Precipitation'])
        
        # 去掉其他 NaN
        neighbors = [x for x in neighbors if pd.notna(x)]
        
        # 計算平均補值
        if len(neighbors) > 0:
            filled.loc[i] = round(np.mean(neighbors), 3)
df_rain['Precipitation'] = filled

# 基礎時間特徵
df['weekday'] = df['Slot_Start'].dt.dayofweek
df['hour'] = df['Slot_Start'].dt.hour
df['minute'] = df['Slot_Start'].dt.minute
df['date'] = df['Slot_Start'].dt.date

df['is_morning_session'] = ((df['hour'] >= 9) & (df['hour'] < 12)).astype(int)
df['is_afternoon_session'] = ((df['hour'] >= 14) & (df['hour'] < 17)).astype(int)

# 週期性編碼
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)

# 根據日期給定當天降水量
for i in tqdm(range(len(df)), desc='設定降雨量並補值'):
    date = str.strip(str(df.loc[i, 'date']))
    rain_row = df_rain[df_rain['date'] == date]
    if not rain_row.empty:
        df.loc[i, 'Precipitation'] = rain_row['Precipitation'].values[0]
    else:
        df.loc[i, 'Precipitation'] = 0.0

# 根據日期給定當天平均溫度
for i in tqdm(range(len(df)), desc='設定平均溫度並補值'):
    date = str.strip(str(df.loc[i, 'date']))
    temp_row = df_temp[df_temp['date'] == date]
    if not temp_row.empty:
        df.loc[i, 'temperature'] = temp_row['temperature'].values[0]
    else:
        df.loc[i, 'temperature'] = df_temp['temperature'].mean()


# 場次進度
def calc_session_progress(row):
    h, m = row['hour'], row['minute']
    if 9 <= h < 12:
        start_min, end_min = 9 * 60, 11 * 60 + 30
    else:
        start_min, end_min = 14 * 60, 16 * 60 + 30
    current_min = h * 60 + m
    return (current_min - start_min) / (end_min - start_min)

df['session_progress'] = df.apply(calc_session_progress, axis=1)


進行特徵工程...


設定平均溫度並補值: 100%|██████████| 602888/602888 [01:38<00:00, 6126.30it/s]



In [6]:
# 區域靜態特徵
print("計算區域靜態特徵...")
zone_total = df_raw.groupby('Zone_ID').size()
zone_rate = zone_total / zone_total.sum()
df['zone_baseline_risk'] = df['Zone_ID'].map(zone_rate).fillna(0)

# 區域在上午/下午的取締比例
df_raw['is_morning'] = ((df_raw['Datetime'].dt.hour >= 9) & (df_raw['Datetime'].dt.hour < 12)).astype(int)
morning_counts = df_raw[df_raw['is_morning'] == 1].groupby('Zone_ID').size()
afternoon_counts = df_raw[df_raw['is_morning'] == 0].groupby('Zone_ID').size()
total_counts = df_raw.groupby('Zone_ID').size()

zone_morning_ratio = (morning_counts / total_counts).fillna(0.5)
zone_afternoon_ratio = (afternoon_counts / total_counts).fillna(0.5)
df['zone_morning_ratio'] = df['Zone_ID'].map(zone_morning_ratio).fillna(0.5)
df['zone_afternoon_ratio'] = df['Zone_ID'].map(zone_afternoon_ratio).fillna(0.5)

# 區域在每個星期幾的風險
df_raw['weekday'] = df_raw['Datetime'].dt.dayofweek
zone_weekday_avg = df_raw.groupby(['Zone_ID', 'weekday']).size().unstack(fill_value=0)
zone_weekday_avg = zone_weekday_avg / (zone_weekday_avg.sum(axis=1).values.reshape(-1, 1) + 1e-10)

def get_zone_weekday_risk(row):
    zone, wd = row['Zone_ID'], row['weekday']
    if zone in zone_weekday_avg.index:
        return zone_weekday_avg.loc[zone, wd]
    return 0.14

df['zone_weekday_risk'] = df.apply(get_zone_weekday_risk, axis=1)

計算區域靜態特徵...


In [7]:
# 歷史特徵
print("計算歷史特徵...")
grouped = df.groupby('Zone_ID')['count_in_slot']
df['lag_1'] = grouped.shift(1).fillna(0)
df['lag_2'] = grouped.shift(2).fillna(0)
df['lag_3'] = grouped.shift(3).fillna(0)
df['lag_4'] = grouped.shift(4).fillna(0)
df['recent_1h_count'] = grouped.shift(1).rolling(window=4, min_periods=1).sum().fillna(0)
df['decay_recent'] = grouped.shift(1).ewm(halflife=2).mean().fillna(0)

# 當天已取締紀錄
print("計算當天已取締紀錄...")
df['date_zone_key'] = df['date'].astype(str) + '_' + df['Zone_ID'].astype(str)
df['today_cumsum'] = df.groupby('date_zone_key')['count_in_slot'].cumsum() - df['count_in_slot']

# 當天全域
df['today_global_cumsum'] = df.groupby('date')['count_in_slot'].cumsum() - df['count_in_slot']


計算歷史特徵...
計算當天已取締紀錄...


In [8]:
# 空間特徵
print("計算空間特徵...")
pivot_lag1 = df.pivot(index='Slot_Start', columns='Zone_ID', values='lag_1').fillna(0)

spatial_features = []
for zone in zone_list:
    neighbors = neighbor_map.get(zone, [])
    neighbors = [n for n in neighbors if n in pivot_lag1.columns]
    
    temp = pd.DataFrame({'Slot_Start': pivot_lag1.index, 'Zone_ID': zone})
    
    if neighbors:
        temp['neighbor_lag1_sum'] = pivot_lag1[neighbors].sum(axis=1).values
        temp['neighbor_lag1_mean'] = pivot_lag1[neighbors].mean(axis=1).values
        temp['neighbor_has_event'] = (pivot_lag1[neighbors] > 0).any(axis=1).astype(int).values
        temp['neighbor_event_count'] = (pivot_lag1[neighbors] > 0).sum(axis=1).values
    else:
        temp['neighbor_lag1_sum'] = 0
        temp['neighbor_lag1_mean'] = 0
        temp['neighbor_has_event'] = 0
        temp['neighbor_event_count'] = 0
    
    spatial_features.append(temp)

df_spatial = pd.concat(spatial_features, ignore_index=True)
df = pd.merge(df, df_spatial, on=['Slot_Start', 'Zone_ID'], how='left')

# 填補空值
spatial_cols = ['neighbor_lag1_sum', 'neighbor_lag1_mean', 'neighbor_has_event', 'neighbor_event_count']
df[spatial_cols] = df[spatial_cols].fillna(0)


計算空間特徵...


## 6.進階特徵工程

In [9]:
print("計算進階特徵...")

# 同星期幾 + 同時段的歷史取締率
df_raw['hour'] = df_raw['Datetime'].dt.hour
zone_weekday_hour = df_raw.groupby(['Zone_ID', 'weekday', 'hour']).size().reset_index(name='hist_count')
zone_weekday_hour_total = df_raw.groupby(['weekday', 'hour']).size().reset_index(name='total_count')
zone_weekday_hour = zone_weekday_hour.merge(zone_weekday_hour_total, on=['weekday', 'hour'])
zone_weekday_hour['zone_weekday_hour_rate'] = zone_weekday_hour['hist_count'] / (zone_weekday_hour['total_count'] + 1)

# 合併到主資料
df = df.merge(
    zone_weekday_hour[['Zone_ID', 'weekday', 'hour', 'zone_weekday_hour_rate']], 
    on=['Zone_ID', 'weekday', 'hour'], 
    how='left'
)
df['zone_weekday_hour_rate'] = df['zone_weekday_hour_rate'].fillna(0)

# 鄰居當天累積取締數
print("計算鄰居當天累積取締...")
pivot_today = df.pivot(index='Slot_Start', columns='Zone_ID', values='today_cumsum').fillna(0)

neighbor_today_features = []
for zone in zone_list:
    neighbors = neighbor_map.get(zone, [])
    neighbors = [n for n in neighbors if n in pivot_today.columns]
    
    temp = pd.DataFrame({'Slot_Start': pivot_today.index, 'Zone_ID': zone})
    
    if neighbors:
        temp['neighbor_today_sum'] = pivot_today[neighbors].sum(axis=1).values
        temp['neighbor_today_max'] = pivot_today[neighbors].max(axis=1).values
    else:
        temp['neighbor_today_sum'] = 0
        temp['neighbor_today_max'] = 0
    
    neighbor_today_features.append(temp)

df_neighbor_today = pd.concat(neighbor_today_features, ignore_index=True)
df = pd.merge(df, df_neighbor_today, on=['Slot_Start', 'Zone_ID'], how='left')
df['neighbor_today_sum'] = df['neighbor_today_sum'].fillna(0)
df['neighbor_today_max'] = df['neighbor_today_max'].fillna(0)

# 交互特徵
print("計算交互特徵...")
df['risk_x_morning'] = df['zone_baseline_risk'] * df['is_morning_session']
df['risk_x_afternoon'] = df['zone_baseline_risk'] * df['is_afternoon_session']
df['risk_x_progress'] = df['zone_baseline_risk'] * df['session_progress']
df['weekday_hour_risk'] = df['zone_weekday_risk'] * df['zone_weekday_hour_rate']

# 時間窗口統計特徵
print("計算時間窗口統計...")
# 過去一週同星期幾同時段的取締次數 (用歷史資料估算)
df['slot_key'] = df['weekday'].astype(str) + '_' + df['hour'].astype(str) + '_' + df['minute'].astype(str)
zone_slot_hist = df_raw.copy()
zone_slot_hist['slot_key'] = zone_slot_hist['weekday'].astype(str) + '_' + zone_slot_hist['Datetime'].dt.hour.astype(str) + '_' + zone_slot_hist['Datetime'].dt.minute.astype(str)
zone_slot_count = zone_slot_hist.groupby(['Zone_ID', 'slot_key']).size().reset_index(name='hist_slot_count')
df = df.merge(zone_slot_count, on=['Zone_ID', 'slot_key'], how='left')
df['hist_slot_count'] = df['hist_slot_count'].fillna(0)

計算進階特徵...
計算鄰居當天累積取締...
計算交互特徵...
計算時間窗口統計...
計算交互特徵...
計算時間窗口統計...


## 7. 預測目標 (預測該場次內是否會被取締)

In [10]:
print("\n 建立預測目標 (該場次剩餘時間內是否取締)...")

# ========== 預測視窗設定 ==========
# 根據實際停車情境：通常停一整個上午或下午 (約 2.5 小時)
# window_size = 10 表示預測未來 2.5 小時 (10 個 15 分鐘 slot)
# 這符合「早上停到中午」或「下午停到放學」的情境
WINDOW_SIZE = 10  # 2.5 小時 = 一個完整場次
# ==================================

print(f"預測視窗: {WINDOW_SIZE} 個時段 = {WINDOW_SIZE * 15} 分鐘 ({WINDOW_SIZE * 15 / 60:.1f} 小時)")

# 使用 Forward-looking window 計算未來 N 個 slot 的取締數
indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=WINDOW_SIZE)
df['future_window_count'] = df.groupby('Zone_ID')['count_in_slot'].rolling(window=indexer, min_periods=1).sum().values
df['label'] = df['future_window_count'].fillna(0).astype(int)
df['relevance'] = df['label'].clip(upper=2)

df_model = df.dropna().copy()
print(f"模型資料: {len(df_model)} 筆")


 建立預測目標 (該場次剩餘時間內是否取締)...
預測視窗: 10 個時段 = 150 分鐘 (2.5 小時)
模型資料: 602888 筆


## 8. 特徵選擇與切分

In [11]:
features = [
    # 基礎時間特徵
    'weekday', 'hour', 'minute',
    'is_morning_session', 'is_afternoon_session',
    'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos',
    'session_progress',
    # 區域靜態特徵
    'zone_baseline_risk', 'zone_morning_ratio', 'zone_afternoon_ratio', 'zone_weekday_risk',
    # 歷史特徵
    'lag_1', 'lag_2', 'lag_3', 'lag_4',
    'recent_1h_count', 'decay_recent',
    'today_cumsum', 'today_global_cumsum',
    # 空間特徵
    'neighbor_lag1_sum', 'neighbor_lag1_mean', 'neighbor_has_event', 'neighbor_event_count',
    'neighbor_today_sum', 'neighbor_today_max',
    # 進階特徵
    'zone_weekday_hour_rate', 'hist_slot_count',
    # 交互特徵
    'risk_x_morning', 'risk_x_afternoon', 'risk_x_progress', 'weekday_hour_risk',
    # 外部特徵
    'Precipitation', 'temperature'
]

print(f"\n使用 {len(features)} 個特徵")
X = df_model[features]
y = df_model['relevance']
X.to_csv("X_df.csv", index=False, encoding="utf-8")
y.to_csv("Y_df.csv", index=False, encoding="utf-8")

# 時間切分
unique_dates = sorted(df_model['date'].unique())
split_idx = int(len(unique_dates) * 0.8)
split_date = unique_dates[split_idx]

mask_train = df_model['date'] < split_date
mask_test = df_model['date'] >= split_date

X_train, y_train = X[mask_train], y[mask_train]
X_test, y_test = X[mask_test], y[mask_test]
meta_test = df_model[mask_test][['Slot_Start', 'Zone_ID', 'label', 'relevance', 'date']].copy()

train_pos_rate = (df_model[mask_train]['label'] > 0).mean()
test_pos_rate = (df_model[mask_test]['label'] > 0).mean()

print(f"訓練資料: ~ {split_date} ({split_idx} 天)")
print(f"測試資料: {split_date} ~ ({len(unique_dates) - split_idx} 天)")
print(f"訓練集正樣本比例: {train_pos_rate:.2%}")
print(f"測試集正樣本比例: {test_pos_rate:.2%}")
print(f"訓練集: {len(X_train)} 筆, 測試集: {len(X_test)} 筆")



使用 36 個特徵
訓練資料: ~ 2025-04-25 (843 天)
測試資料: 2025-04-25 ~ (211 天)
訓練集正樣本比例: 8.54%
測試集正樣本比例: 8.50%
訓練集: 482196 筆, 測試集: 120692 筆
訓練資料: ~ 2025-04-25 (843 天)
測試資料: 2025-04-25 ~ (211 天)
訓練集正樣本比例: 8.54%
測試集正樣本比例: 8.50%
訓練集: 482196 筆, 測試集: 120692 筆


## 9. 訓練模型 (二元分類)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import xgboost as xgb

print("\n準備多模型訓練...")

# 將 label 轉為二元 (有取締 = 1, 無取締 = 0)
y_train_binary = (y_train > 0).astype(int)
y_test_binary = (y_test > 0).astype(int)

print(f"訓練集正樣本: {y_train_binary.sum()} ({y_train_binary.mean():.2%})")
print(f"測試集正樣本: {y_test_binary.sum()} ({y_test_binary.mean():.2%})")

# 計算 scale_pos_weight 來處理不平衡
scale_pos_weight = (y_train_binary == 0).sum() / (y_train_binary == 1).sum()
print(f"scale_pos_weight: {scale_pos_weight:.2f}")

# ==========================================
# 1. XGBoost
# ==========================================
print("\n🚀 訓練 XGBoost...")
dtrain = xgb.DMatrix(X_train, label=y_train_binary)
dtest = xgb.DMatrix(X_test, label=y_test_binary)

# 優化後的參數
params_clf = {
    'objective': 'binary:logistic',
    'eval_metric': ['auc', 'logloss'],
    'eta': 0.0410,  # 降低學習率
    'max_depth': 8,  # 稍微增加深度
    'min_child_weight': 16,  # 增加以減少過擬合
    'subsample': 0.6875,
    'colsample_bytree': 0.5556,
    'colsample_bylevel': 0.7,
    'reg_alpha': 0.0047,  # 增加 L1 正則化
    'reg_lambda': 0.0090,  # 增加 L2 正則化
    'gamma': 0.4218,  # 加入剪枝
    'scale_pos_weight': scale_pos_weight,
    'tree_method': 'hist',
    'seed': 42
}

print("----------------------------")
model = xgb.train(
    params_clf,
    dtrain,
    num_boost_round=800,  # 增加輪數
    evals=[(dtrain, 'train'), (dtest, 'eval')],
    early_stopping_rounds=50,
    verbose_eval=50
)
print("----------------------------")

# 預測機率作為風險分數
meta_test['risk_score_xgb'] = model.predict(dtest)

# ==========================================
# 2. Logistic Regression
# ==========================================
print("\n📏 訓練 Logistic Regression...")
# 線性模型通常需要特徵標準化
model_lr = make_pipeline(
    StandardScaler(),
    LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42)
)
model_lr.fit(X_train, y_train_binary)
meta_test['risk_score_lr'] = model_lr.predict_proba(X_test)[:, 1]

# ==========================================
# 3. Random Forest
# ==========================================
print("\n🌲 訓練 Random Forest...")
model_rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    min_samples_leaf=4,
    class_weight='balanced',
    n_jobs=-1,
    random_state=42
)
model_rf.fit(X_train, y_train_binary)
meta_test['risk_score_rf'] = model_rf.predict_proba(X_test)[:, 1]

# 預設使用 XGBoost 作為主要輸出 (兼容後續代碼)
meta_test['risk_score'] = meta_test['risk_score_xgb']


準備多模型訓練...
訓練集正樣本: 41192 (8.54%)
測試集正樣本: 10263 (8.50%)
scale_pos_weight: 10.71

🚀 訓練 XGBoost...
----------------------------

訓練集正樣本: 41192 (8.54%)
測試集正樣本: 10263 (8.50%)
scale_pos_weight: 10.71

🚀 訓練 XGBoost...
----------------------------
[0]	train-auc:0.79301	train-logloss:0.68247	eval-auc:0.76716	eval-logloss:0.68230
[0]	train-auc:0.79301	train-logloss:0.68247	eval-auc:0.76716	eval-logloss:0.68230
[50]	train-auc:0.86007	train-logloss:0.49929	eval-auc:0.82984	eval-logloss:0.49008
[50]	train-auc:0.86007	train-logloss:0.49929	eval-auc:0.82984	eval-logloss:0.49008
[100]	train-auc:0.87628	train-logloss:0.46530	eval-auc:0.83190	eval-logloss:0.45829
[100]	train-auc:0.87628	train-logloss:0.46530	eval-auc:0.83190	eval-logloss:0.45829
[150]	train-auc:0.88863	train-logloss:0.44661	eval-auc:0.83191	eval-logloss:0.44414
[150]	train-auc:0.88863	train-logloss:0.44661	eval-auc:0.83191	eval-logloss:0.44414
[200]	train-auc:0.90062	train-logloss:0.42916	eval-auc:0.83018	eval-logloss:0.43219
[200]	tra

## 10. 評估

In [20]:
def precision_at_k(df_res, score_col, k=5):
    precisions = []
    for slot, group in df_res.groupby('Slot_Start'):
        if group['label'].sum() > 0:
            top_k = group.nlargest(k, score_col)
            hits = (top_k['label'] > 0).sum()
            precisions.append(hits / k)
    return np.mean(precisions) if precisions else 0

def hit_rate_at_k(df_res, score_col, k=3):
    hits = 0
    total = 0
    for slot, group in df_res.groupby('Slot_Start'):
        if group['label'].sum() > 0:
            total += 1
            top_k = group.nlargest(k, score_col)
            if (top_k['label'] > 0).any():
                hits += 1
    return hits / total if total > 0 else 0

def calc_ndcg_per_slot(df_res, score_col, k=10):
    ndcg_list = []
    for slot, group in df_res.groupby('Slot_Start'):
        if group['label'].sum() > 0 and len(group) > 1:
            y_true = group['label'].values.reshape(1, -1)
            y_score = group[score_col].values.reshape(1, -1)
            try:
                ndcg_list.append(ndcg_score(y_true, y_score, k=k))
            except:
                pass
    return np.mean(ndcg_list) if ndcg_list else 0

def safe_zone_accuracy(df_res, score_col, bottom_k=5):
    accuracies = []
    for slot, group in df_res.groupby('Slot_Start'):
        bottom_k_zones = group.nsmallest(bottom_k, score_col)
        safe_count = (bottom_k_zones['label'] == 0).sum()
        accuracies.append(safe_count / bottom_k)
    return np.mean(accuracies)

def danger_zone_hit_rate(df_res, score_col, top_k=5):
    """Top K 區域中有取締的比例"""
    hit_rates = []
    for slot, group in df_res.groupby('Slot_Start'):
        top_k_zones = group.nlargest(top_k, score_col)
        hit_count = (top_k_zones['label'] > 0).sum()
        hit_rates.append(hit_count / top_k)
    return np.mean(hit_rates)

# 計算指標
active_slots = meta_test.groupby('Slot_Start')['label'].sum()
active_slot_count = (active_slots > 0).sum()
total_slots = len(active_slots)

print(f"\n📊 資料統計:")
print(f"   🔹 區域數: {len(zones)}")
print(f"   🔹 測試時段數: {total_slots}")
print(f"   🔹 有取締的時段: {active_slot_count} ({active_slot_count/total_slots:.1%})")

models_to_eval = {
    'XGBoost': 'risk_score_xgb',
    'Logistic Regression': 'risk_score_lr',
    'Random Forest': 'risk_score_rf'
}

for model_name, score_col in models_to_eval.items():
    print(f"\n{'='*20} {model_name} 評估 {'='*20}")
    
    print(f"\n📊 Precision@K:")
    print(f"   🔹 Precision@3: {precision_at_k(meta_test, score_col, 3):.2%}")
    print(f"   🔹 Precision@5: {precision_at_k(meta_test, score_col, 5):.2%}")

    print(f"\n📊 Hit Rate@K (在有取締的時段，Top K 至少命中一個的比例):")
    print(f"   🔹 Hit Rate@3: {hit_rate_at_k(meta_test, score_col, 3):.2%}")
    print(f"   🔹 Hit Rate@5: {hit_rate_at_k(meta_test, score_col, 5):.2%}")
    print(f"   🔹 Hit Rate@10: {hit_rate_at_k(meta_test, score_col, 10):.2%}")

    print(f"\n📊 排名品質:")
    print(f"   🔹 NDCG@5: {calc_ndcg_per_slot(meta_test, score_col, 5):.4f}")
    print(f"   🔹 NDCG@10: {calc_ndcg_per_slot(meta_test, score_col, 10):.4f}")

    # 計算隨機基準線
    random_baseline = 1 - test_pos_rate  # 隨機選的安全率 = 負樣本比例

    print(f"\n📊 安全區 vs 危險區 (模型區分能力):")
    print(f"   🔹 隨機選 5 個的安全率 (基準線): {random_baseline:.2%}")
    print(f"   🔹 模型 Bottom 5 安全率: {safe_zone_accuracy(meta_test, score_col, 5):.2%}")
    print(f"   🔹 模型 Top 5 安全率: {1 - danger_zone_hit_rate(meta_test, score_col, 5):.2%}")
    print(f"   🔹 模型 Top 5 危險率: {danger_zone_hit_rate(meta_test, score_col, 5):.2%}")
    print(f"   ────────────────────────────────")
    safe_bottom = safe_zone_accuracy(meta_test, score_col, 5)
    safe_top = 1 - danger_zone_hit_rate(meta_test, score_col, 5)
    print(f"   📈 區分能力 = {safe_bottom:.2%} - {safe_top:.2%} = {(safe_bottom - safe_top):.2%}")


📊 資料統計:
   🔹 區域數: 22
   🔹 測試時段數: 5486
   🔹 有取締的時段: 2669 (48.7%)

==================== XGBoost 評估 ====================

📊 Precision@K:
   🔹 Precision@3: 34.15%
   🔹 Precision@3: 34.15%
   🔹 Precision@5: 30.26%

📊 Hit Rate@K (在有取締的時段，Top K 至少命中一個的比例):
   🔹 Precision@5: 30.26%

📊 Hit Rate@K (在有取締的時段，Top K 至少命中一個的比例):
   🔹 Hit Rate@3: 68.79%
   🔹 Hit Rate@3: 68.79%
   🔹 Hit Rate@5: 80.14%
   🔹 Hit Rate@5: 80.14%
   🔹 Hit Rate@10: 92.77%

📊 排名品質:
   🔹 Hit Rate@10: 92.77%

📊 排名品質:
   🔹 NDCG@5: 0.3817
   🔹 NDCG@5: 0.3817
   🔹 NDCG@10: 0.4715

📊 安全區 vs 危險區 (模型區分能力):
   🔹 隨機選 5 個的安全率 (基準線): 91.50%
   🔹 NDCG@10: 0.4715

📊 安全區 vs 危險區 (模型區分能力):
   🔹 隨機選 5 個的安全率 (基準線): 91.50%
   🔹 模型 Bottom 5 安全率: 95.76%
   🔹 模型 Bottom 5 安全率: 95.76%
   🔹 模型 Top 5 安全率: 85.28%
   🔹 模型 Top 5 安全率: 85.28%
   🔹 模型 Top 5 危險率: 14.72%
   ────────────────────────────────
   🔹 模型 Top 5 危險率: 14.72%
   ────────────────────────────────
   📈 區分能力 = 95.76% - 85.28% = 10.48%

==================== Logistic Regression 評估 ===========

## 11. 儲存模型

In [14]:
model.save_model('parking_risk_model_v4_zone.json')

# 儲存區域名稱對照表
zone_info = []
for zone_id in zones:
    zone_name = zone_names.get(zone_id, f'Zone_{zone_id}')
    zone_locs = df_rules[df_rules['Zone_ID'] == zone_id]['Original_Location'].tolist()
    zone_info.append({
        'Zone_ID': zone_id,
        'Zone_Name': zone_name,
        'Locations': ', '.join(zone_locs)
    })

pd.DataFrame(zone_info).to_csv('zone_info.csv', index=False, encoding='utf-8-sig')

print("\n模型已儲存: parking_risk_model_v4_zone.json")
print("區域資訊已儲存: zone_info.csv")



模型已儲存: parking_risk_model_v4_zone.json
區域資訊已儲存: zone_info.csv


## 13. 範例輸出

In [15]:
# ========== 自訂時間設定 ==========
# 設定為 None 則使用測試集最新時段，或指定特定時間
# 格式: "YYYY-MM-DD HH:MM" (會自動對齊到 15 分鐘)
# 例如: CUSTOM_SAMPLE_TIME = "2024-10-15 09:30"
CUSTOM_SAMPLE_TIME = "2025-11-21 09:00"  # 修改這裡來指定時間
# ==================================

if CUSTOM_SAMPLE_TIME is not None:
    # 使用自訂時間
    custom_dt = pd.to_datetime(CUSTOM_SAMPLE_TIME)
    sample_slot = custom_dt.floor('15min')
    
    # 檢查是否在測試集範圍內
    if sample_slot in meta_test['Slot_Start'].values:
        sample_data_base = meta_test[meta_test['Slot_Start'] == sample_slot].copy()
        print(f"\n✅ 使用自訂時間: {CUSTOM_SAMPLE_TIME}")
    else:
        print(f"\n⚠️ 自訂時間 {CUSTOM_SAMPLE_TIME} 不在測試集範圍內")
        print(f"   測試集範圍: {meta_test['Slot_Start'].min()} ~ {meta_test['Slot_Start'].max()}")
        print("   改用測試集最新時段...")
        latest_date = meta_test['date'].max()
        latest_data = meta_test[meta_test['date'] == latest_date]
        sample_slot = latest_data['Slot_Start'].min()
        sample_data_base = latest_data[latest_data['Slot_Start'] == sample_slot].copy()
else:
    # 使用測試集最新時段
    latest_date = meta_test['date'].max()
    latest_data = meta_test[meta_test['date'] == latest_date]
    sample_slot = latest_data['Slot_Start'].min()
    sample_data_base = latest_data[latest_data['Slot_Start'] == sample_slot].copy()

print(f"\n⏰ 時段: {sample_slot}")

models_to_show = {
    'XGBoost': 'risk_score_xgb',
    'Logistic Regression': 'risk_score_lr',
    'Random Forest': 'risk_score_rf'
}

for model_name, score_col in models_to_show.items():
    print(f"\n{'='*20} {model_name} 預測結果 {'='*20}")
    
    # 複製並排序
    sample_data = sample_data_base.copy()
    sample_data = sample_data.sort_values(score_col, ascending=False)

    print(f"\n🔴 最危險 Top 10 區域:")
    for i, (_, row) in enumerate(sample_data.head(10).iterrows()):
        zone_name = zone_names.get(row['Zone_ID'], f"Zone_{row['Zone_ID']}")
        actual = "⚠️ 有取締" if row['label'] > 0 else ""
        print(f"   {i+1}. {zone_name:<15} 分數: {row[score_col]:.4f} {actual}")

    print(f"\n🟢 最安全 Top 10 區域:")
    for i, (_, row) in enumerate(sample_data.tail(10).iloc[::-1].iterrows()):
        zone_name = zone_names.get(row['Zone_ID'], f"Zone_{row['Zone_ID']}")
        actual = "⚠️ 有取締" if row['label'] > 0 else ""
        print(f"   {i+1}. {zone_name:<15} 分數: {row[score_col]:.4f} {actual}")

    # 該時段的統計
    top5_danger = sample_data.head(5)
    bot5_safe = sample_data.tail(5)
    print(f"\n📈 此時段統計 ({model_name}):")
    print(f"   🔹 Top 5 危險區命中: {(top5_danger['label'] > 0).sum()}/5 ({(top5_danger['label'] > 0).mean():.0%})")
    print(f"   🔹 Bottom 5 安全區正確: {(bot5_safe['label'] == 0).sum()}/5 ({(bot5_safe['label'] == 0).mean():.0%})")
    print(f"   🔹 此時段總取締區域數: {(sample_data['label'] > 0).sum()}/{len(sample_data)}")


✅ 使用自訂時間: 2025-11-21 09:00

⏰ 時段: 2025-11-21 09:00:00

==================== XGBoost 預測結果 ====================

🔴 最危險 Top 10 區域:
   1. 電資附近            分數: 0.9275 ⚠️ 有取締
   2. 翠亨宿舍西           分數: 0.9169 
   3. L停附近            分數: 0.9151 ⚠️ 有取締
   4. 武嶺裡面            分數: 0.8470 
   5. 理工區             分數: 0.8090 ⚠️ 有取締
   6. 武嶺外面            分數: 0.7780 
   7. 理學院             分數: 0.7475 
   8. 藝術學院            分數: 0.6232 ⚠️ 有取締
   9. 海工館             分數: 0.6218 
   10. 翠亨宿舍東           分數: 0.5897 

🟢 最安全 Top 10 區域:
   1. 運動場區            分數: 0.0080 
   2. 生科附近            分數: 0.2157 
   3. 圖資區             分數: 0.3526 
   4. 翠嶺道             分數: 0.3654 
   5. 行政大樓            分數: 0.4024 
   6. 文學院             分數: 0.4200 
   7. 管院              分數: 0.4266 
   8. 海科院             分數: 0.4733 
   9. 活動中心區           分數: 0.5094 
   10. 社科院             分數: 0.5109 

📈 此時段統計 (XGBoost):
   🔹 Top 5 危險區命中: 3/5 (60%)
   🔹 Bottom 5 安全區正確: 5/5 (100%)
   🔹 此時段總取締區域數: 5/22

==================== Logistic Regression 預測結果

## 14. R² 與模型解釋力分析

In [21]:
from sklearn.metrics import r2_score, brier_score_loss, log_loss, roc_auc_score
from sklearn.calibration import calibration_curve
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import explained_variance_score

print("=" * 70)
print("📊 多模型 R² 與解釋力分析")
print("=" * 70)

y_test_np = y_test_binary.values
null_prob = y_train_binary.mean()

models_to_analyze = {
    'XGBoost': 'risk_score_xgb',
    'Logistic Regression': 'risk_score_lr',
    'Random Forest': 'risk_score_rf'
}

for model_name, score_col in models_to_analyze.items():
    print(f"\n\n{'#'*30}")
    print(f"   模型: {model_name}")
    print(f"{'#'*30}")
    
    # 取得該模型的預測分數
    y_pred_np = meta_test[score_col].values

    # ==========================================
    # 1. 核心指標：AUC-ROC
    # ==========================================
    auc_score = roc_auc_score(y_test_np, y_pred_np)
    print(f"\n📈 核心分類指標:")
    print(f"   🔹 AUC-ROC: {auc_score:.4f}")

    # ==========================================
    # 2. 機率校準分析
    # ==========================================
    print(f"\n📈 機率校準分析:")
    print(f"   🔹 預測機率平均: {y_pred_np.mean():.2%}")
    prob_diff = abs(y_test_np.mean() - y_pred_np.mean())
    if prob_diff > 0.05:
        print(f"   ⚠️ 機率未校準 (差距 {prob_diff:.2%})")
    else:
        print(f"   ✅ 機率校準良好")

    # ==========================================
    # 3. 排名相關性 (Spearman)
    # ==========================================
    correlations = []
    for slot, group in meta_test.groupby('Slot_Start'):
        if len(group) > 5 and group['label'].sum() > 0:
            pred_rank = group[score_col].rank(ascending=False)
            actual_rank = group['label'].rank(ascending=False, method='average')
            corr, _ = spearmanr(pred_rank, actual_rank)
            if not np.isnan(corr):
                correlations.append(corr)

    avg_spearman = np.mean(correlations) if correlations else 0
    rank_r2 = avg_spearman ** 2

    print(f"\n📈 排名相關性 (Spearman):")
    print(f"   🔹 平均 Spearman ρ: {avg_spearman:.4f}")
    print(f"   🔹 排名決定係數 ρ²: {rank_r2:.4f}")

    # ==========================================
    # 4. 區域層級分析 (Pearson)
    # ==========================================
    zone_stats = meta_test.groupby('Zone_ID').agg({
        score_col: 'mean',
        'label': 'mean'
    }).reset_index()

    zone_corr, zone_p = pearsonr(zone_stats['label'], zone_stats[score_col])
    zone_r2 = zone_corr ** 2

    print(f"\n📈 區域層級分析:")
    print(f"   🔹 區域 Pearson 相關: {zone_corr:.4f}")
    print(f"   🔹 區域相關 R²: {zone_r2:.4f}")

    # ==========================================
    # 5. 分組校準 (Calibration by Decile)
    # ==========================================
    meta_temp = meta_test.copy()
    meta_temp['pred_decile'] = pd.qcut(meta_temp[score_col], q=10, labels=False, duplicates='drop')
    calibration_df = meta_temp.groupby('pred_decile').agg({
        score_col: 'mean',
        'label': ['mean', 'count']
    }).round(4)
    calibration_df.columns = ['預測機率', '實際正率', '樣本數']
    
    pred_by_group = calibration_df['預測機率'].values
    actual_by_group = calibration_df['實際正率'].values
    if len(pred_by_group) > 2:
        group_corr, _ = pearsonr(pred_by_group, actual_by_group)
        group_r2 = group_corr ** 2
    else:
        group_r2 = 0

    # ==========================================
    # 總結表格
    # ==========================================
    # 評價邏輯
    eval_auc = '✅ 優秀' if auc_score > 0.8 else ('⚠️ 普通' if auc_score > 0.7 else '❌ 差')
    eval_spearman = '✅ 有效' if avg_spearman > 0.2 else '⚠️ 弱'
    eval_pearson = '✅ 顯著' if zone_p < 0.05 else '❌ 不顯著'
    eval_calib = '✅ 單調' if group_r2 > 0.8 else '⚠️ 待改進'

    print(f"""
    ┌────────────────────────────────────────────────────────────────┐
    │  指標 ({model_name:<19})│  數值      │  評價              │
    ├────────────────────────────────────────────────────────────────┤
    │  AUC-ROC (分類能力)            │  {auc_score:>6.4f}    │  {eval_auc:<16}  │
    │  Spearman ρ (排名相關)         │  {avg_spearman:>6.4f}    │  {eval_spearman:<16}  │
    │  區域 Pearson r                │  {zone_corr:>6.4f}    │  {eval_pearson:<16}  │
    │  分組校準 R²                   │  {group_r2:>6.4f} │ {eval_calib:<16} │
    └────────────────────────────────────────────────────────────────┘
    """)

📊 多模型 R² 與解釋力分析


##############################
   模型: XGBoost
##############################

📈 核心分類指標:
   🔹 AUC-ROC: 0.8005

📈 機率校準分析:
   🔹 預測機率平均: 23.72%
   ⚠️ 機率未校準 (差距 15.21%)

📈 排名相關性 (Spearman):
   🔹 平均 Spearman ρ: 0.2251
   🔹 排名決定係數 ρ²: 0.0507

📈 區域層級分析:
   🔹 區域 Pearson 相關: 0.4897
   🔹 區域相關 R²: 0.2399

    ┌────────────────────────────────────────────────────────────────┐
    │  指標 (XGBoost            )│  數值      │  評價              │
    ├────────────────────────────────────────────────────────────────┤
    │  AUC-ROC (分類能力)            │  0.8005    │  ✅ 優秀              │
    │  Spearman ρ (排名相關)         │  0.2251    │  ✅ 有效              │
    │  區域 Pearson r                │  0.4897    │  ✅ 顯著              │
    │  分組校準 R²                   │  0.8604 │ ✅ 單調             │
    └────────────────────────────────────────────────────────────────┘
    


##############################
   模型: Logistic Regression
##############################

📈 核心分類指標:
   🔹 AUC-ROC: 0.7594

📈 機率校準分析: